In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from scipy import sparse
from scipy.sparse import coo_matrix
from scipy.sparse import coo_array
from scipy.sparse import identity
from scipy.sparse import diags

import fockspy
from fockspy import fock_classes as fc

In [ ]:
# параметры теста

# первая подсистема

s1 =  1
use_loc_1 = 1
n_max_1 = 4
m1 = 3
loc_1_ = (1, 3, 2)

# вторая подсистема

s2 =  1
use_loc_2 = 0
n_max_2 = 7
m2 = 7
loc_2_ = (2, 3, 1, 4, 5, 1, 2)

if s1 > 0:
    st1 = 'Bose'
else:
    st1 = 'Fermi'

if s2 > 0:
    st2 = 'Bose'
else:
    st2 = 'Fermi'

if s1 > 0 and use_loc_1 > 0:
    m1_ = loc_1_
else:
    m1_ = m1

if s2 > 0 and use_loc_2 > 0:
    m2_ = loc_2_
else:
    m2_ = m2

f1 = fc.fock_space(m1_, n_max_1, st1)
f2 = fc.fock_space(m2_, n_max_2, st2)

f = fc.fock_space_kron(f1,f2)  

In [ ]:
# проверка расчета номера состояния и заселенностей

import random

nn = 1000

for i in range(0, nn):

    j = random.randint(0, f.dimension - 1)

    n = f.occupations(j)

    j_ = f.index(np.array(n))

    if j != j_:

        print('error')

        break

print('done')

In [ ]:
# проверить операторы уничтожения

import random
import math

a = f.annihilate

nn = 1000

for i in range(0, nn):

    j = random.randint(0, f.dimension - 1)
    k = random.randint(0, f.modes - 1)

    psi = np.zeros(f.dimension, dtype = complex)
    psi[j]=1

    psi_ = a[k] @ psi

    j_ = (np.where(abs(psi_) > 0))[0]

    n = np.array(f.occupations(j))

    if n[k] == 0:

        if j_.size > 0:

            print('error')
            break

    else:

        j_ = j_[0]
        n_ = np.array(f.occupations(j_))

        n_[k] = n_[k] + 1

        n_ = n_ - n

        if not np.all((n_ == 0)):

            print('error')
            break

        w = psi_[j_]

        p = 1.0

        if  k < m1:
            p = p * s1**sum(n[0:k])
        else:
            p = p * s2**sum(n[m1:k])

            if s1 < 0 and s2 < 0:
                p = p * (-1)**sum(n[0:m1])

        if not w == math.sqrt(n[k]) * p:

            print('error')
            break

print('done')

In [ ]:
#проверить операторы рождения

import random
import math

a_dag = f.create

nn = 1000

for i in range(0, nn):

    j = random.randint(0, f.dimension - 1)
    k = random.randint(0, f.modes - 1)

    psi = np.zeros(f.dimension, dtype = complex)
    psi[j]=1

    psi_ = a_dag[k] @ psi

    j_ = (np.where(abs(psi_) > 0))[0]

    n = np.array(f.occupations(j))

    nt1 = n[0:m1].sum()
    nt2 = n[m1:].sum()
    
    if (nt1 > n_max_1 or nt2 > n_max_2):

        print('error')
        break
    
    


    if (k < m1 and nt1 == n_max_1) or (k >= m1 and nt2 == n_max_2) \
        or (s1 < 0 and k < m1 and n[k] == 1) or (s2 < 0 and k >= m1 and n[k] == 1) \
            or (k < m1 and s1 > 0 and use_loc_1 > 0 and loc_1_[k] == n[k]) \
                or (k >= m1 and s2 > 0 and use_loc_2 > 0 and loc_2_[k - m1] == n[k]):

        if j_.size > 0:

            print('error')
            break

    else:

        j_ = j_[0]
        n_ = np.array(f.occupations(j_))

        n_[k] = n_[k] - 1

        n_ = n_ - n

        if not np.all((n_ == 0)):

            print('error')
            break

        w = psi_[j_]

        p = 1.0

        if  k < m1:
            p = p * s1**sum(n[0:k])
        else:
            p = p * s2**sum(n[m1:k])

            if s1 < 0 and s2 < 0:
                p = p * (-1)**sum(n[0:m1])

        if not w == math.sqrt(n[k] + 1) * p:

            print('error')
            break

print('done')